Ce tutoriel reprend celui sur le site de [scikit learn](https://docs.google.com/spreadsheets/d/15x0AUKgeUqtcExbmE3eHlNjaE_WFhaNpB9dQEgmVevc/edit#gid=0)

# I Analyse du problème

Nous allons utilisé un dataset inclu dans scikit learn.

Il contient une série de texte classé par catégorie. Nous allons nous ateler à une tâche de classification supervisée dont le but sera de retrouver cette catégorie.

Nous commençons par selectionner un sous ensemble de ces catégories pour que le dataset ne soit pas trop important:

In [1]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

On importe ensuite le jeu de données correspondant à ces catégories.

In [2]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

On peut afficher les premières lignes d'un des fichiers:

In [3]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


# II Choix de l'encodage et du pre processing

## A. Théorie

Scikit-learn implémente différent encodage ([doc](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text)):
- CountVectorizer: Transforme une série de documents en une matrice de comptage (Bag of Word)
- HashingVectorizer: Transforme une série de documents en une matrice d'occurence
- TfidfVectorizer: Transforme une série de documents en une matrice de fréquence

Nous appliquerons dans un premier temps CountVectorizer()

Dans scikit learn, le pre-processing est réalisé via les paramètres de la classe d'encodage.
On a en effet les paramètres suivants (entre parenthèse la valeur par défaut):
- strip_accents (FALSE): supprime tous les accents (réduit la taille du vocabulaire)
- lowercase (TRUE): convertie tous les caractères (réduit la taille du vocabulaire)
- stop_words (None): permet d'ajouter une liste de stop word ou d'utiliser une liste par défaut {english}
- max_df (1.0): ignore les mots qui ont une fréquence trop élevée dans le dataset (autre manière de gérer les stopwords)
- min_df (1): ignore les mots présents moins de fois que le seuil.
- analyzer (word) : Qu'est ce qui constitue une token (une unité de texte) {‘word’, ‘char’, ‘char_wb’} or callable (callable permet d'utiliser un preprocessing externe)
- ngram_range (1,1): taille du n-gram à analyser
- vocabulary: permet de proposer un vocabulaire précis

Nous n'avons pas toutes les options de pre processing comme ce qui est inclu dans les packages comme nltk ou spacy mais nous verons plus tard qu'il sera possible de les utilser dans scikit learn.

## B. Pratique

Nous créons donc la matrice de comptage:

In [4]:
# Dans notre jeu d'entrainement il y a 2257 documents
print(f"nombre de documents dans le jeu d'entrainement: {len(twenty_train.data)}")

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
# On commence avec un preprocessing minimal (lower, token = word)
X_train_counts = count_vect.fit_transform(twenty_train.data)
print(f"dimension de la matrice de comptage: {X_train_counts.shape}")

nombre de documents dans le jeu d'entraonement: 2257
dimension de la matrice de comptage: (2257, 35788)


D'après la dimension de la matrice de comptage on déduit qu'il y a 35788 mots dans notre vocabulaire

In [5]:
# on peut retrouver l'indexe de chaque mot
print(count_vect.vocabulary_.get(u'algorithm'))
print(count_vect.vocabulary_.get(u'computer'))
print(count_vect.vocabulary_.get(u'good'))
print(count_vect.vocabulary_.get(u'the'))

# plus un mot est fréquent plus sont indexe est elevé 

4690
9338
15576
32142


# III Entainement et prédiction

Il ne reste plus qu'à entrainer un modèle qui prendra pour features la matrice construite et pour target le label de chacun des documents

In [6]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, twenty_train.target)

In [7]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

In [8]:
X_test_counts = count_vect.transform(twenty_test.data)
predicted = clf.predict(X_test_counts)

In [9]:
from sklearn.metrics import accuracy_score
accuracy_score(twenty_test.target, predicted)

0.9340878828229028

In [10]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted, target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.92      0.90      0.91       319
         comp.graphics       0.95      0.95      0.95       389
               sci.med       0.96      0.91      0.93       396
soc.religion.christian       0.91      0.97      0.94       398

              accuracy                           0.93      1502
             macro avg       0.93      0.93      0.93      1502
          weighted avg       0.93      0.93      0.93      1502



In [11]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
predicted = clf.predict(X_new_counts)
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


# IV Tuning du pre processing

On crée pour cela un Pipeline

In [12]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB()),
])

In [13]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'vect__strip_accents': [None, 'ascii'],
    'vect__stop_words': ["english", None],
    'vect__max_df': [1, 0.9,0.8,0.7],
}

In [14]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
# on ne l'entraine que sur une partie du jeu de données
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

In [17]:
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.9778427486607931
vect__max_df: 0.9
vect__ngram_range: (1, 1)
vect__stop_words: 'english'
vect__strip_accents: None


In [18]:
predicted_grid = gs_clf.predict(twenty_test.data)

In [19]:
print(metrics.classification_report(twenty_test.target, predicted_grid, target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.93      0.91      0.92       319
         comp.graphics       0.95      0.96      0.96       389
               sci.med       0.95      0.93      0.94       396
soc.religion.christian       0.93      0.96      0.95       398

              accuracy                           0.94      1502
             macro avg       0.94      0.94      0.94      1502
          weighted avg       0.94      0.94      0.94      1502



# V Preprocessing avec NLTK

In [26]:
import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
vect_nltk = CountVectorizer(tokenizer=LemmaTokenizer()) 

[nltk_data] Downloading package punkt to /Users/charles/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [22]:
text_clf_nltk = Pipeline([
    ('vect', vect),
    ('clf', MultinomialNB()),
])

In [31]:
nltk.download('punkt') 
nltk.download('wordnet')
nltk.download('omw-1.4')
gs_clf_nltk = text_clf_nltk.fit(twenty_train.data, twenty_train.target)

In [32]:
predicted_nltk = gs_clf_nltk.predict(twenty_test.data)

In [33]:
print(metrics.classification_report(twenty_test.target, predicted_nltk, target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.90      0.88      0.89       319
         comp.graphics       0.92      0.94      0.93       389
               sci.med       0.93      0.89      0.91       396
soc.religion.christian       0.92      0.95      0.94       398

              accuracy                           0.92      1502
             macro avg       0.92      0.91      0.91      1502
          weighted avg       0.92      0.92      0.92      1502



Ici les résultats sont moins bon mais nous n'avons pas tuné les modèles.

# VI Exercice

Nous allons découvrir tous les modèles suivant au travers de l'exemple d'une compétition kaggle.
L'objectif de cette [compétition](https://www.kaggle.com/competitions/quora-insincere-questions-classification/data?select=train.csv) est de déterminer si des questions postées sur le site Quora sont sincères ou non.

Consignes:
- Commencez par lire les consignes de la compétition pour lire les enjeux, repérer notament la métrique d'interet
- Télécharger seulement le train set, nous travaillerons dessus.
- Mettez en place un MLflow avec un projet spécifique. Pour tous les modèles que vous lancerez faites remonter spécifiquement le F1 score et des tags correspondant au modèle (quel préprocessing (steaming, url hangling), quel encodage (tf, tfidf, count vect, Word2Vect...), quel modèle (regresison, réseau de neurones)). Si vous lancez plusieurs modèles d'une meme catégorie pensez également à remonter les hyperparamètres modifiés. 
- Sur le jeu de données, lancez en vous aidant de scikit learn:
    - un count vectoriser
    - un tf
    - un tf idf
 Vous avez le choix du modèle, vous pouvez meme si vous avez le temps faire un pycaret entre votre base encodée et votre target et tuner les hyperparamètres.
 - Intégrez ensuite si vous avez le temps du preprocessing avec NLTK.
